In [1]:
from build_epi_ga_df import *
import pandas as pd
from patsy import dmatrices
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


data_dir = "../data/erin_data"

ga_data, epi_data = build_epi_ga_data(data_dir)

drop_cols = ["age_at_ga", "n_extractions", "n_primary_extract",
             "n_perm_extract", "has_bib1kc", "res6mcage_ga", "Epi", "GATx",
             "ddscareindex", "describe_health_q", "has_diagnosis_q",
             "hospital_admission_q", "looked_after_child"]

epi_data.drop(drop_cols, axis=1, inplace=True)
epi_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354 entries, 200 to 1404
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   decayed_teeth            354 non-null    float64
 1   missing_teeth            354 non-null    float64
 2   filled_teeth             354 non-null    float64
 3   dmft                     354 non-null    float64
 4   res6mcage                354 non-null    float64
 5   res6mcbetterstart        354 non-null    object 
 6   gender                   354 non-null    object 
 7   father_age_at_q          111 non-null    float64
 8   mother_age_at_q          295 non-null    float64
 9   on_benefits              294 non-null    object 
 10  ethnicity                345 non-null    object 
 11  english_additional_lang  349 non-null    object 
 12  special_ed_needs         350 non-null    object 
 13  father_highest_ed        294 non-null    object 
 14  mother_highest_ed      

In [2]:
mother_english = epi_data.mother_ethnicity == "White British"
moved_is_na = epi_data.age_mother_moved_uk.isna()
change_cols = mother_english & moved_is_na
epi_data.loc[change_cols, "age_mother_moved_uk"] = 0

fill_values = {
 "father_age_at_q": epi_data.father_age_at_q.mean(),
 "mother_age_at_q": epi_data.mother_age_at_q.mean(),
 "on_benefits": "Don't know",
 "ethnicity": "Other",
 "english_additional_lang": "Don't know",
 "special_ed_needs": "Don't know",
 "father_highest_ed": "Don't know",
 "mother_highest_ed": "Don't know",
 "mother_ethnicity": "Other",
 "age_father_complete_ed": "Don't know",
 "father_birthplace": "Other",
 "imd_2010_decile": epi_data.imd_2010_decile.median(),
 "socio_economic_pos": "Don't know",
 "age_mother_moved_uk": epi_data.age_mother_moved_uk.median(),
 "mother_birthplace": "Other",
 "questionaire_language": "English",
}

for feature, value in fill_values.items():
    epi_data[feature] = epi_data[feature].fillna(value)

epi_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354 entries, 200 to 1404
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   decayed_teeth            354 non-null    float64
 1   missing_teeth            354 non-null    float64
 2   filled_teeth             354 non-null    float64
 3   dmft                     354 non-null    float64
 4   res6mcage                354 non-null    float64
 5   res6mcbetterstart        354 non-null    object 
 6   gender                   354 non-null    object 
 7   father_age_at_q          354 non-null    float64
 8   mother_age_at_q          354 non-null    float64
 9   on_benefits              354 non-null    object 
 10  ethnicity                354 non-null    object 
 11  english_additional_lang  354 non-null    object 
 12  special_ed_needs         354 non-null    object 
 13  father_highest_ed        354 non-null    object 
 14  mother_highest_ed      

In [3]:
X_train, X_test, _, _ = train_test_split(epi_data.drop("dmft", axis=1),
                                                       epi_data.dmft,
                                                       test_size=0.33,
                                                       random_state=42)
train_idxs = X_train.index
test_idxs = X_test.index

drop_cols = ["decayed_teeth", "missing_teeth", "filled_teeth"]
train_dat = epi_data.loc[train_idxs].drop(drop_cols, axis=1)
test_dat = epi_data.loc[test_idxs].drop(drop_cols, axis=1)

In [15]:
features = train_dat.drop("dmft", axis=1).columns
patsy_string = "dmft ~ "
patsy_string += " + ".join(features)
y, X = dmatrices(patsy_string, data=train_dat, return_type="dataframe")

top_3_models = pd.DataFrame({
    "features": [],
    "rsquared": [],
    "f": []
})

def score_epi_model(features, show_summary=False):
    mod = sm.OLS(y, X[features])
    res = mod.fit()
    if show_summary:
        print(res.summary())
    return res.rsquared_adj, res.fvalue

model_has_improved = True
loop_no = 0
while model_has_improved:
    model_has_improved = False
    loop_no += 1

    top_models_this_loop = {
        "features": [],
        "rsquared": [],
        "f": []
    }

    if loop_no == 1:
        for feature in X.columns:
            rsquared, f = score_epi_model([feature])
            top_models_this_loop["features"].append([feature])
            top_models_this_loop["rsquared"].append(rsquared)
            top_models_this_loop["f"].append(f)
    else:
        for test_feature in X.columns:
            for extra_features in top_3_models.features:
                if test_feature in extra_features:
                    continue
                model_features = [test_feature] + extra_features
                rsquared, f = score_epi_model(model_features)
                if rsquared > top_3_models.rsquared.max():
                    top_models_this_loop["features"].append(model_features)
                    top_models_this_loop["rsquared"].append(rsquared)
                    top_models_this_loop["f"].append(f)


    if len(top_models_this_loop["features"]):
        model_has_improved = True
        top_3_models = (pd.DataFrame(top_models_this_loop)
                        .sort_values("rsquared", ascending=False)
                        .iloc[:3, :])

    print("=" * 80)
    print(f"Loop Number {loop_no} - start")
    print(f"Top 3 Models")
    print(top_3_models)

Loop Number 1 - start
Top 3 Models
             features  rsquared           f
52        [res6mcage]  0.305884  105.441610
53  [father_age_at_q]  0.304639  104.830317
54  [mother_age_at_q]  0.289031   97.347869
Loop Number 2 - start
Top 3 Models
                                             features  rsquared          f
63         [questionaire_language[T.Urdu], res6mcage]  0.348942  64.511315
64   [questionaire_language[T.Urdu], father_age_at_q]  0.348917  64.504443
14  [english_additional_lang[T.Yes], father_age_at_q]  0.342097  62.617641
Loop Number 3 - start
Top 3 Models
                                             features  rsquared          f
23  [mother_highest_ed[T.Don't know], questionaire...  0.375696  48.541002
22  [mother_highest_ed[T.Don't know], questionaire...  0.375551  48.511560
40  [socio_economic_pos[T.Don't know], questionair...  0.375062  48.412485
Loop Number 4 - start
Top 3 Models
                                             features  rsquared          f
24  [moth

In [58]:
best_features = top_3_models.iloc[0,:].features
patsy_string = "dmft ~ "
patsy_string += " + ".join(best_features)
y, X = dmatrices(patsy_string,
                 data=epi_data,
                 return_type="dataframe")
epi_data.loc[train_idxs]

,decayed_teeth,missing_teeth,filled_teeth,dmft,res6mcage,res6mcbetterstart,gender,father_age_at_q,mother_age_at_q,on_benefits,...,father_highest_ed,mother_highest_ed,mother_ethnicity,age_father_complete_ed,father_birthplace,imd_2010_decile,socio_economic_pos,age_mother_moved_uk,mother_birthplace,questionaire_language
1155,0.0,0.0,0.0,0.0,66.0,No,Male,250.0,234.0,No,...,5 GCSE equivalent,A-level equivalent,White British,17-19,United Kingdom,1.0,Employed not mat dep,NaN,England,English
1325,3.0,0.0,0.0,3.0,66.0,No,Male,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1169,0.0,0.0,0.0,0.0,66.0,Yes,Female,NaN,303.0,Yes,...,5 GCSE equivalent,Higher than A-level,Pakistani,NaN,NaN,1.0,Employed not mat dep,NaN,Pakistan,English
1082,0.0,0.0,0.0,0.0,66.0,Yes,Female,NaN,268.0,No,...,Don't know,5 GCSE equivalent,Pakistani,NaN,NaN,3.0,Benefits but coping,NaN,Pakistan,English
1087,0.0,0.0,0.0,0.0,66.0,Yes,Male,NaN,382.0,Yes,...,Higher than A-level,5 GCSE equivalent,Pakistani,NaN,NaN,1.0,Employed no access to money,23.0,Pakistan,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,0.0,0.0,0.0,0.0,66.0,No,Male,431.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1157,0.0,0.0,0.0,0.0,66.0,No,Male,554.0,469.0,Yes,...,<5 GCSE equivalent,<5 GCSE equivalent,White British,17-19,United Kingdom,2.0,Employed no access to money,NaN,England,English
1321,0.0,2.0,0.0,2.0,60.0,No,Female,NaN,411.0,No,...,Other,A-level equivalent,White British,NaN,NaN,1.0,Employed not mat dep,NaN,England,English
1399,9.0,0.0,0.0,9.0,66.0,No,Male,NaN,378.0,Yes,...,Other,<5 GCSE equivalent,Pakistani,NaN,NaN,2.0,Benefits but coping,16.0,Pakistan,English


In [60]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 201 to 1396
Data columns (total 34 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Intercept                                 37 non-null     float64
 1   special_ed_needs[T.Not SEN]               37 non-null     float64
 2   special_ed_needs[T.SEN Support]           37 non-null     float64
 3   mother_birthplace[T.England]              37 non-null     float64
 4   mother_birthplace[T.India]                37 non-null     float64
 5   mother_birthplace[T.Other]                37 non-null     float64
 6   mother_birthplace[T.Pakistan]             37 non-null     float64
 7   mother_birthplace[T.Philippines]          37 non-null     float64
 8   mother_birthplace[T.Poland]               37 non-null     float64
 9   mother_birthplace[T.Scotland]             37 non-null     float64
 10  mother_birthplace[T.Sri Lanka]      

In [42]:
mod = sm.OLS(y, X.loc[train_idxs])
res = mod.fit()
print(res.summary())

1057    10.000000
1045     6.032286
1279     1.000000
1391     6.999315
1363     4.029547
dtype: float64